In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/players.csv')
players.head()

In [ ]:
players.shape

In [ ]:
#checking for any NA
players.isnull().sum()

In [ ]:
#Allows me to use ILOC to filter data
df = pd.DataFrame(players)
df.head(1303)

In [ ]:
#filtering to only view secondary defensive players
DB_pos = ['CB','SS','FS','DB']
df.loc[df['position'].isin(DB_pos)]
df = df.loc[df['position'].isin(DB_pos)]
df

In [ ]:
#visualizing defensive backs broken down by college name
plt.figure(figsize=(24,8))
graph = sns.countplot(x='collegeName',data=df,palette='cividis',
                      order=df.collegeName.value_counts().iloc[:15].index)

for p in graph.patches:
    height = p.get_height()
    graph.text(p.get_x()+p.get_width()/2., height + 0.3,height ,ha="center")
    
plt.show()

In [ ]:
#import week 10 data
week10 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/week10.csv')

In [ ]:
week10.head()

In [ ]:
week10.isnull().sum()

In [ ]:
week10.drop(['dir','time','x','y','a','dis','o','event','jerseyNumber','frameId','gameId','playId','playDirection'],axis=1)
week10 = week10.drop(['dir','time','x','y','a','dis','o','event','jerseyNumber','frameId','gameId','playId','playDirection'],axis=1)
week10

In [ ]:
#drop NAN
week10.dropna()

In [ ]:
df1 = pd.DataFrame(week10)
df1

In [ ]:
#viewing only defensive backs
DB_Pos = ['DB','SS','FS','CB']
df1.loc[df1['position'].isin(DB_Pos)]
df1 = df1.loc[df1['position'].isin(DB_Pos)]
df1

In [ ]:
# The average acceleration and speed for Defensive Back during week 10
df1.groupby('displayName').mean('s')
df1 = df1.groupby('displayName').mean('s')
df1

In [ ]:
# Scatter Plot representing acceleration on 'y' axis and speed on 'x' axis
ax1 = df1.plot.scatter(x='s', y='a', c='Green')

In [ ]:
display(df,df1)

In [ ]:
#changing index to be able to merge 
df1.reset_index(inplace = True)
df1

In [ ]:
#both data sets combined
dfcombine = pd.merge(df,df1)
dfcombine

In [ ]:
#AVG Weight is 199.183673
dfcombine.describe()

In [ ]:
#Modeling Portion
#Code implemented will include linear regressions with weight and speed as independent variables and acceleration as dependent variable. 


In [ ]:
x = dfcombine[['weight','s']]
y = dfcombine['a']
x

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [ ]:
#Base Regression 70% split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3)

In [ ]:
#create a model and fit
reg.fit(x_train,y_train)

In [ ]:
#print the coefficients for each column and/or feature of the model

print(reg.coef_)

In [ ]:
#predictons on the test data
y_pred = reg.predict(x_test)
y_pred


In [ ]:
#Actual output observed
y_test

In [ ]:
#model accuracy measured by MSE
(np.mean((y_pred - y_test)**2))

In [ ]:
#Linear Regression looking at weight as the dependent variable

x1 = dfcombine[['s']]
y1 = dfcombine['weight']
x1

In [ ]:
reg1 = LinearRegression()

In [ ]:
#Split of 75/25 between train and test

x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1,test_size = 0.25)

In [ ]:
#train using training data

reg1.fit(x1_train,y1_train)

In [ ]:
#display coefficients

print(reg1.coef_)

In [ ]:
#print y test data predictions
y1_pred = reg1.predict(x1_test)
y1_pred
                    

In [ ]:
y1_test

In [ ]:
#Checking accuracy with MSE
mean_squared_error(y1_test, y1_pred)

In [ ]:
##Random Forest Classifier Model with weight as dependent
##Train 70% Test 30%
xforest = dfcombine[['s','height']]
yforest = dfcombine['weight']


In [ ]:
#split of 70/30 (train/test)

from sklearn.model_selection import train_test_split
xforest_train, xforest_test, yforest_train, yforest_test = train_test_split(xforest,yforest,test_size= 0.3)


In [ ]:
#Import RFC using 80 trees

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(80)
model.fit(xforest_train, yforest_train)

In [ ]:
model.score(xforest_test, yforest_test)

In [ ]:
yforest_predicted = model.predict(xforest_test)
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(yforest_test, yforest_predicted)
cm1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(20,10))
sn.heatmap(cm1, annot=True)
plt.xlabel('prediction')
plt.ylabel('result')
